Imports
---

You will need a few non-standard files:

- `mesh_neu.py` reads in a mesh into `V` and `E`
- `trimesh.py` will plot a mesh with `V` and `E`
- `vtk_writer.py` will write to a `.vtu` file with cell data

In [1]:
import mesh_neu
import trimesh
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse as sparse
import vtk_writer
import refine_mesh
%matplotlib inline

(1) Check the orientation and read the mesh
---

First we'll write a function so that for *each* cell consider a matrix 
$$
\left|
\begin{array}{3}
1 & x_0 & y_0\\
1 & x_1 & y_1\\
1 & x_2 & y_2
\end{array}
\right|
$$
which will give the orientation of a cell

In [2]:
def checkorientation(V, E):
    sgn = np.zeros((E.shape[0],))
    
    for i in range(E.shape[0]):
        xi = V[E[i, :],0]
        yi = V[E[i, :],1]
        A = np.zeros((3,3))
        A[:,0] = 1.0
        A[:,1] = xi
        A[:,2] = yi
        
        sgn[i] = np.linalg.det(A)
    return sgn

Next read both `V` and `E`.  Then check the oreintation and swap two column if the orientation is negative (clockwise). (*note*: the meshes are already oriented.  this is more for completeness)

In [3]:
# read mesh
V, E = mesh_neu.read_neu('square2.neu')

# refine mesh
#V, E = refine_mesh.refine2dtri(V, E)

sgn = checkorientation(V, E)
I = np.where(sgn<0)[0]
E1 = E[I,1]
E2 = E[I,2]
E[I,2] = E1
E[I,1] = E2

Now set the mesh elements.  So `ne` is the number of elements and `nv` is the number of vertices.  `X` are the $x$-values and `Y` are $y$-values.

In [4]:
ne = E.shape[0]
nv = V.shape[0]
X = V[:,0]
Y = V[:,1]

(2) Set up the arrays
---

You'll need a few arrays.  The first are the normal vectors *per element*:

- `nx` is $3\times$`ne` and is the $x$-component of the normal for each edge
- `ny` is $3\times$`ne` and is the $y$-component of the normal for each edge

You'll also need the height of the triangle, *per edge* which is defined as `h`, which is $3\times$`ne`.

Then you'll need the centers of the elements:

- `cx` is `ne`$\times 1$ and is $x$-coordinate of each element
- `cy` is `ne`$\times 1$ and is $y$-coordinate of each element

In [6]:
# TODO: create arrays
nx =  np.zeros((3,ne))
ny =  np.zeros((3,ne))
h =   np.zeros((3,ne))
cx =  np.zeros(ne)
cy =  np.zeros(ne)

For each element this loop will

- set the coordinates `xi` and `yi`
- set the centers
- determine the edgelengths
- compute the area
- set the `nx`, `ny`, and `h` values for each edge `0`, `1`, and `2` of the current element

Then the normal vectors are normalized.

In [ ]:
for i in range(ne):
    # TODO: set array values
    # (for cell i, set nx, ny, cx, cy, and h)
    nx = 
    
# normalize
nlength = np.sqrt(nx**2 + ny**2)
nx = nx / nlength
ny = ny / nlength

# more metrics
hinv = 1.0 / h

(3) Construct Element Relationships
---

This step will construct (as in class):

- `E2E`: a list of element-to-element connections
- `V2V`: a list of vertex-to-vertex connections

In [ ]:
# construct vertex to vertex graph
ID = np.kron(np.arange(0, ne), np.ones((3,)))
G = sparse.coo_matrix((np.ones((ne*3,)), (E.ravel(), ID,)))
E2E = G.T * G
V2V = G * G.T

Now a neiborhood list is constructed.  For each element in `E`, the vertices of each neighboring element (from `E2E`) are checked for maching vertices.  Each face will have a neighboring element.  Boundary faces will retain a `-1`.

In [ ]:
Enbrs = -np.ones((ne,3), dtype=int)
for i in range(ne):
    vi = E[i, :]
    
    nbrids = np.where(E2E[i, :].data == 2)[0]
    nbrs = E2E[i, :].indices[nbrids]

    # for each nbr, find the face it goes with
    for j in nbrs:
        vj = E[j, :]
        
        # edge 0
        if (vi[0] in vj) and (vi[1] in vj):
            Enbrs[i, 0] = j
        # edge 1
        if (vi[1] in vj) and (vi[2] in vj):
            Enbrs[i, 1] = j
        # edge 2
        if (vi[2] in vj) and (vi[0] in vj):
            Enbrs[i, 2] = j

(4) Set initial Values
---

Here you set the inital values for $u$, $v$, and $p$.

In [ ]:
# TODO: set initial values
u = 
v = 
p = 

You can write out the inital value as `output.vtu` in the following call.  Then you can verify the initial condition in Paraview.

In [ ]:
p[np.where(np.abs(p)<1e-15)[0]] = 0.0  # trim small values for Paraview
vtk_writer.write_basic_mesh(V, E, cdata=p, mesh_type='tri')

(5) Set up "right" elements and "left" elements.
---

In this step we're going to set up a list of "this" elements, called `mapL`.  These will be the element where you'll compute on.  Then you'll need a list of neighboring elements, called `mapR`.

Also, set a list of boundary elements in `mapB`.

In [ ]:
mapR = Enbrs.T

# find boundary elements and set mapR
ids = np.where(mapR.ravel()==-1)[0]
r, c = np.where(mapR==-1)
mapR = mapR.ravel()
mapR[ids] = c
mapR = mapR.reshape((3,ne))

# set boundary
mapB = ids.copy()
vmapB = c

# set mapL to be "this"
mapL = np.outer(np.ones((3,), dtype=int), np.arange(0,ne, dtype=int))

(6) Time Step
---

Here you will time step according to the notes.

In [ ]:
# set the time step
dt = 0.25 * h.min()
t = 0

# set the number of time steps
nstep=1000

for tstep in range(nstep):
    
    print("tstep %d of %d" % (tstep,nstep))
    uL = u[mapL]
    uR = u[mapR]
    vL = v[mapL]
    vR = v[mapR]
    pL = p[mapL]
    pR = p[mapR]
    
    # set the boundary conditions
    shp = uR.shape
    uR = uR.ravel()
    uR[mapB] = -uL.ravel()[mapB]
    uR = uR.reshape(shp)
    
    vR = vR.ravel()
    vR[mapB] = -vL.ravel()[mapB]
    vR = vR.reshape(shp)
    
    pR = pR.ravel()
    pR[mapB] = pL.ravel()[mapB]
    pR = pR.reshape(shp)
    
    # TODO: set the flux

    
    # TODO: set the update
    u = u + dt* # SUM OVER EDGES flux / h
    v = v + dt* # SUM OVER EDGES flux / h
    p = p + dt* # SUM OVER EDGES flux / h
    
    if (tstep % 10) == 0:
        p[np.where(np.abs(p)<1e-15)[0]] = 0.0
        vtk_writer.write_basic_mesh(V, E, cdata=p, mesh_type='tri', fname='p%04d.vtu'%(tstep,))
    
    t = t+dt

In [ ]:
# TODO: plots